In [3]:
import pandas as pd
data = pd.read_csv("carsdata_features.csv", engine='c')
len(data)

64553

In [4]:
import numpy as np
cat_var = data.select_dtypes(include=['int64'])
cont_var = data.select_dtypes(include=['float64'])

# Clustering

Les variables sont un mélange de variable catégorielles et variables continues. Les algorithmes de clustering tels que ** k-means ** reposent sur une définition de la distance (par exemple euclidienne) qui s'appliquent uniquement aux variables continues. Pour pouvoir inclure les variables catégorielles, on peut utiliser une variation de k-means qui s'appelle ** k-prototypes **.
Les étapes:
1. Imputer les valeurs NaN avec la moyenne de la colonne
2. Standardiser les variables continues
3. Définir une mesure de la qualité de clustering
4. Itérations de k-prototypes avec différents nombres de clusters pour optimiser la mesure de clustering.

## 1. Imputer les valeurs manquantes


In [5]:
import numpy as np
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(np.array(cont_var))

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [6]:
cont_var_array = imp.transform(np.array(cont_var))

In [7]:
#avant imputation
np.array(cont_var)[:4,:1]

array([[ 86. ],
       [ 72.2],
       [ 89.9],
       [  nan]])

In [8]:
#apres imputation
cont_var_array[:4,:1]

array([[ 86.       ],
       [ 72.2      ],
       [ 89.9      ],
       [ 80.8939467]])

# 2. Standardiser les variables continues
Le champ de variation de chaques variables doit etre le meme pour que le poids relatif de chaque variable puisse etre equvalent dans la mesure de distance des échantillons. La standardisation consiste en une soustraction de la moyenne et une division par la standard deviation.

In [9]:
from sklearn import preprocessing
scaled_cont_var_array = preprocessing.scale(cont_var_array)
scaled_cont_var_array[:4,:2]

array([[ 0.98232348,  0.07834041],
       [-1.67257714, -1.75224355],
       [ 1.73262149,  2.06502068],
       [ 0.        ,  0.        ]])

# Définir une mesure de qualité du clustering
Pour définir la qualité du clustering on utilisa le silouhette index:
\begin{equation*} \frac{b - a}{max(a,b)}\end{equation*} 

Avec a: la distance moyenne d'un point avec tous les autres point de la meme classe,
et b : la distance moyenne d'un point avec tous les points d'un autre cluster le plus proche.

Pour les variables catégorielles on ne peut pas utiliser la distance euclidienne. On utilise dans ce cas la distance de Hamming qui compte le nombre de diffrences sur toutes les variables catégorielles pour deux échantillons. Les valeurs a et b seront donc une combinaison de distance euclidienne et de Hamming avec un facteur gamma correspondant au ratio du nombres de variable dans les deux catégories.



In [19]:
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=DeprecationWarning)
    kmeans_model = MiniBatchKMeans(n_clusters=3,init='k-means++').fit(scaled_cont_var_array[:10,:])


In [20]:
labels = kmeans_model.labels_
metrics.silhouette_score(scaled_cont_var_array[:10,:], labels)

0.64700392094601944

In [15]:
labels[:10]

array([1, 2, 0, 1, 0, 1, 1, 1, 1, 1])